In [ ]:
# TRAINING CODE WITH CENTRALIZED FILE NAMING

import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras._tf_keras.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger, ModelCheckpoint
from keras._tf_keras.keras.applications.efficientnet_v2 import EfficientNetV2S # Import the corresponding application
from keras._tf_keras.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation
from keras._tf_keras.keras.models import Model
from keras._tf_keras.keras.optimizers import Adam

# 🔹 Set model name in one place
file_name = "EfficientNetV2S"  # Change this to rename all outputs

# Dataset paths
TRAIN_DIR = '' # Directory of the Train Data Set
VAL_DIR = '' # Directory of the Validation Data Set

if not os.path.exists(TRAIN_DIR) or not os.path.exists(VAL_DIR):
    raise FileNotFoundError("Dataset directories not found!")

# Data generators
train_data_generator = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    horizontal_flip=True
)

val_data_generator = ImageDataGenerator(rescale=1.0 / 255)

train_set = train_data_generator.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)

val_set = val_data_generator.flow_from_directory(
    VAL_DIR,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

# Build model with logits and softmax layer separated
base_model = EfficientNetV2S(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu')(x)
logits_layer = Dense(6, activation=None, name='logits')(x)
softmax_output = Activation('softmax', name='predictions')(logits_layer)

model = Model(inputs=base_model.input, outputs=softmax_output)

# Compile with label smoothing
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    optimizer=Adam(1e-4),
    metrics=['accuracy']
)

# Callbacks using centralized file_name
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1),
    CSVLogger(f'{file_name}_log.csv'),
    ModelCheckpoint(f'{file_name}_best.keras', save_best_only=True, monitor='val_accuracy', mode='max')
]

# Train the model
epochs = 100
history = model.fit(
    train_set,
    epochs=epochs,
    validation_data=val_set,
    validation_steps=len(val_set),
    steps_per_epoch=len(train_set),
    callbacks=callbacks
)

# Save the final model
model.save(f"{file_name}_final.keras")

# Plot loss and accuracy
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()


In [ ]:
# CALIBRATED MODEL TESTING
# LOAD THE MODEL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.src.saving.saving_api import load_model
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras import backend as K

# 🔹 1. Define temperature scaling function (match model's custom Lambda)
TEMPERATURE = 0.751  # Replace with your learned temperature if different

def temperature_scaling(logits):
    return logits / tf.constant(TEMPERATURE, dtype=tf.float32)

# 🔹 2. Load calibrated model
model = load_model("/home/amir/projects/TRAININGCODE/effnet_calibrated_scipy.keras", custom_objects={"temperature_scaling": temperature_scaling})

# 🔹 3. Prepare test data
TEST_DIR = "/home/amir/projects/BALANCED/val"  # Make sure this path is correct

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_set = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=1,  # Predict per image for detailed confidence
    class_mode='categorical',
    shuffle=False
)

# 🔹 4. Run predictions
probs = model.predict(test_set, verbose=1)
predicted_classes = np.argmax(probs, axis=1)
confidence_scores = np.max(probs, axis=1)
true_classes = test_set.classes
filenames = test_set.filenames
class_indices = {v: k for k, v in test_set.class_indices.items()}

# 🔹 5. Map numeric labels to class names
true_labels = [class_indices[i] for i in true_classes]
pred_labels = [class_indices[i] for i in predicted_classes]

# 🔹 6. Save results to CSV
df = pd.DataFrame({
    "filename": filenames,
    "true_label": true_labels,
    "predicted_label": pred_labels,
    "confidence": confidence_scores
})

df.to_csv("test_results_with_confidence.csv", index=False)
print("✅ Results saved to 'test_results_with_confidence.csv'.")

# Optional: Print top 5 least confident predictions
print("\n🔍 Top 5 least confident predictions:")
print(df.sort_values(by="confidence").head(5))


In [ ]:
# MODEL CALIBRRATION INTEGRATION
# LOAD THE MODEL
# LOOK FOR OPTIMAL TEMPERATURE


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras._tf_keras.keras.models import load_model


# Load saved model
# Define the same temperature used during calibration
TEMPERATURE = 1.007

# Recreate the Lambda function used in the model
def temperature_layer(x):
    return x / TEMPERATURE
def temperature_scaling(logits):
    return logits / tf.constant(TEMPERATURE, dtype=tf.float32)


model = load_model("/home/amir/projects/TRAININGCODE/EfficientNetV2S_calibrated_scipy.keras", custom_objects={"temperature_scaling": temperature_scaling})


# Validation generator (same as training setup)
VAL_DIR = r"" # Directory of the Validation Data Set
val_data_generator = ImageDataGenerator(rescale=1./255)
val_set = val_data_generator.flow_from_directory(
    VAL_DIR,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False  # Important for correct label order
)

# Get true labels and predictions
true_labels = val_set.classes
predictions = model.predict(val_set)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
class_names = list(val_set.class_indices.keys())

print(f"Overall Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names, digits=4))

# Confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

steps = len(val_set)
predictions = model.predict(val_set, steps=steps, verbose=1)
calibrated_probs = predictions

from netcal.presentation import ReliabilityDiagram
from netcal.metrics import ECE

# 🔹 Compute ECE
ece = ECE(bins=15)
ece_score = ece.measure(calibrated_probs, true_labels)
print(f"📊 Calibrated ECE: {ece_score:.4f}")

# 🔹 Plot Reliability Diagram
diagram = ReliabilityDiagram(bins=15)
diagram.plot(calibrated_probs, true_labels)
plt.title("Reliability Diagram (After Temperature Scaling)")
plt.show()

In [ ]:
# MODEL CALIBRATION
# LOAD THE MODEL

import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

from keras.src.saving.saving_api import load_model
from netcal.metrics import ECE
from netcal.presentation import ReliabilityDiagram
from keras.src.legacy.preprocessing.image import ImageDataGenerator
# CHANGE THE MODEL
from keras.src.models.model import Model
from keras._tf_keras.keras.applications.efficientnet_v2 import EfficientNetV2S

TEMPERATURE = 0.751

# 🔹 Load validation set
VAL_DIR = '/home/amir/projects/BALANCED/val'
val_data_generator = ImageDataGenerator(rescale=1.0 / 255)

val_set = val_data_generator.flow_from_directory(
    VAL_DIR,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

# 🔹 Load trained model and extract logits layer
model = load_model("/home/amir/projects/TRAININGCODE/EfficientNetV2S_FocalLoss_best.keras", compile=False)
logits_layer = model.get_layer("logits").output  # Layer with activation=None
logits_model = Model(inputs=model.input, outputs=logits_layer)


# 🔹 Gather all validation data
val_images, val_labels = [], []
for _ in range(len(val_set)):
    x, y = next(val_set)
    val_images.append(x)
    val_labels.append(y)

val_images = np.concatenate(val_images)
val_labels = np.concatenate(val_labels)

# 🔹 Get logits (pre-softmax outputs)
val_logits = logits_model.predict(val_images, verbose=0)

# 🔹 Softmax with temperature function
def softmax_with_temperature(logits, temperature):
    scaled_logits = logits / temperature
    exp_logits = np.exp(scaled_logits - np.max(scaled_logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

# 🔹 Define temperature scaling loss
def temperature_loss(temperature):
    temperature = temperature[0]
    probs = softmax_with_temperature(val_logits, temperature)
    log_probs = np.log(np.clip(probs, 1e-15, 1.0))
    loss = -np.sum(val_labels * log_probs) / val_labels.shape[0]
    return loss

# 🔹 Optimize temperature
opt_result = minimize(temperature_loss, x0=[1.0], bounds=[(0.5, 10.0)], method='L-BFGS-B')
optimal_temperature = opt_result.x[0]
print(f"✅ Optimal Temperature: {optimal_temperature:.4f}")

# 🔹 Calibrate probabilities
calibrated_probs = softmax_with_temperature(val_logits, optimal_temperature)
confidences = np.max(calibrated_probs, axis=1)
pred_labels = np.argmax(calibrated_probs, axis=1)
true_labels = np.argmax(val_labels, axis=1)

# 🔹 Compute ECE
ece = ECE(bins=15)
ece_score = ece.measure(calibrated_probs, true_labels)
print(f"📊 Calibrated ECE: {ece_score:.4f}")

# 🔹 Plot Reliability Diagram
diagram = ReliabilityDiagram(bins=15)
diagram.plot(calibrated_probs, true_labels)
plt.title("Reliability Diagram (After Temperature Scaling)")
plt.show()

import tensorflow as tf
from keras._tf_keras.keras.applications.resnet50 import ResNet50 # Import the corresponding application
from keras.src.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Lambda, Activation
from keras.src.models import Model

# 🔹 Rebuild calibrated model with temperature scaling
input_layer = Input(shape=(224, 224, 3))
base_model = EfficientNetV2S(include_top=False, weights='imagenet', input_tensor=input_layer)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu')(x)
x = Dense(6, activation=None)(x)

# 🔹 Apply learned temperature
from keras.src.layers import Lambda
from keras import backend as K



def temperature_scaling(logits):
    return logits / K.constant(TEMPERATURE)

x = Lambda(temperature_scaling, name="temperature_scaled", output_shape=(6,))(x)


output = Activation('softmax')(x)
calibrated_model = Model(inputs=input_layer, outputs=output)

# 🔹 Load weights from original model
for i, layer in enumerate(calibrated_model.layers):
    try:
        layer.set_weights(model.layers[i].get_weights())
    except:
        pass

# 🔹 Save calibrated model
calibrated_model.save("EfficientNetV2S_calibrated_scipy.keras")
print("📦 Saved calibrated model with temperature scaling.")


In [ ]:
# MODEL CONVERSION
# LOAD THE MODEL
# CHANGE THE CORRESPONDING TEMPERATURE

import tensorflow as tf
import numpy as np

# 🔹 1. Define temperature scaling function (used in your model)
TEMPERATURE = 1.007  # Replace with your learned temperature if different

def temperature_scaling(logits):
    return logits / tf.constant(TEMPERATURE, dtype=tf.float32)

# 🔹 2. Load the temperature-scaled model
model = tf.keras.models.load_model(
    "/home/amir/projects/TRAININGCODE/EfficientNetV2S_calibrated_scipy.keras",
    custom_objects={"temperature_scaling": temperature_scaling}
)

print("✅ Model loaded successfully.")

# 🔹 3. Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: optimize for size/performance
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Optional: ensure float32 output (important for calibrated confidence outputs)
converter.target_spec.supported_types = [tf.float32]

# Perform conversion
tflite_model = converter.convert()

# 🔹 4. Save the TFLite model
tflite_path = "effnet_calibrated.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite model saved as '{tflite_path}'.")


In [ ]:
#CONVERTED MODEL TESTING

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from PIL import Image
import os

# 🔹 1. Load TFLite model
interpreter = tf.lite.Interpreter(model_path="") # Define Model File Directory
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape'][1:3]  # Should be (224, 224)
input_dtype = input_details[0]['dtype']

# 🔹 2. Load test set using ImageDataGenerator
TEST_DIR = "" # Directory of the Test Data Set

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=input_shape,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# 🔹 3. Run inference with TFLite model
predicted_classes = []
confidence_scores = []

print("🔍 Running TFLite inference on test set...")

for i in range(len(test_generator)):
    img, label = test_generator[i]
    
    interpreter.set_tensor(input_details[0]['index'], img.astype(input_dtype))
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    predicted_class = np.argmax(output[0])
    confidence = np.max(output[0])

    predicted_classes.append(predicted_class)
    confidence_scores.append(confidence)

print("✅ Inference complete.")

# 🔹 4. Save results
filenames = test_generator.filenames
true_classes = test_generator.classes
class_indices = {v: k for k, v in test_generator.class_indices.items()}

true_labels = [class_indices[i] for i in true_classes]
pred_labels = [class_indices[i] for i in predicted_classes]

df = pd.DataFrame({
    "filename": filenames,
    "true_label": true_labels,
    "predicted_label": pred_labels,
    "confidence": confidence_scores
})

df.to_csv("tflite_test_results.csv", index=False)
print("📁 Results saved to 'tflite_test_results.csv'.")

# 🔍 Top 5 least confident predictions
print("\n🔍 Top 5 least confident predictions:")
print(df.sort_values(by="confidence").head(5))
